# Autoencoders

<a target="_blank" href="https://colab.research.google.com/github/GaiaSaveri/intro-to-ml/blob/main/notebooks/Lab-13.Autoencoders.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

An autoencoder is a type of neural network architecture that finds the function mapping the features $x$ to itself. This objective is known as reconstruction, and is accomplished through an encoder and a decoder:
* the encoder compresses the input data into a lower-dimensional representation that captures the most important features or patterns;
* the decoder reconstructs the original data from the compressed representation.

The compressed representation learned by an autoencoder can be visualized as a lower-dimensional space, where similar inputs are mapped to nearby points and dissimilar inputs are mapped to distant points. This makes autoencoders useful for tasks such as dimensionality reduction, anomaly detection, and generative modeling.

**NOTE:** *If linear activations are used, or only a single sigmoid hidden layer, then the optimal solution to an autoencoder is strongly related to principal component analysis (PCA). However, the potential of autoencoders resides in their non-linearity, allowing the model to learn more powerful generalizations compared to PCA, and to reconstruct the input with significantly lower information loss.*

In [ ]:
# import all needed packages 
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from time import time
import random

# reproducibility
torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

In [ ]:
# Set arguments
batch_size = 128

# DataLoader Train
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True,
    transform=torchvision.transforms.ToTensor())

trainloader = data.DataLoader(trainset, batch_size=batch_size, shuffle=True)


# DataLoader Test
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True,
    transform=torchvision.transforms.ToTensor())

testloader = data.DataLoader(testset, batch_size=16, shuffle=False)

## Training

##### **1. Define the model architecture of an AutoEncoder** 
 As mentioned, it is divided in two parts: encoder and decoder.  
  The encoder takes as input a flattened image of size $28x28$ and applies a series of fully connected layers with Tanh activation function. The output of its last layer has dimension $2$.   
  The decoder takes the 2-dimensional encoded representation and applies a series of fully connected layers with Tanh activation function in the reverse order. The last layer applies a sigmoid activation function to map the reconstructed output to the range $[0,1]$.

In [ ]:
# YOUR CODE

In [ ]:
def train_model(model, num_epochs, trainloader, criterion, optimizer):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = model.to(device)

    losses = []
   
    for epoch in range(num_epochs):
        train_running_loss = 0.0

        # Set the model to training mode
        model = model.train()
        start = time()
        ## training step
        for i, (images, labels) in enumerate(trainloader):
            
            # difference here! Similar to .flatten()
            images = images.view(-1, 28*28)

            images = images.to(device)
            labels = labels.to(device)

            ## forward + backprop + loss
            codes, decoded = model(images)
            loss = criterion(decoded, images)

            # Reset the gradients to zero
            optimizer.zero_grad()

            loss.backward()

            ## update model params
            optimizer.step()

            train_running_loss += loss.item()
        
            
        losses.append(train_running_loss / i)
        
        model.eval()
        print(f"Epoch: {epoch+1} | Loss: {train_running_loss / i:.4f} | Time: {time()-start:.2f}") 
    
    return losses

For image data with AutoEncoders it is common to use the mean squared error (MSE) loss function, as it measures the difference between the input image pixels and the reconstructed ones. We are not concerned with labels in this case.

In [ ]:
# Optimizer and loss function
model = AutoEncoder()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [ ]:
L = train_model(model, 15, trainloader, criterion, optimizer)

## Test effects
Now that we have trained the AutoEncoder model, let’s take a look at the picture we restored from the compression. Does it look like the original picture?

In [ ]:
# Show images function
def show_images(images):
    sqrtn = int(np.ceil(np.sqrt(images.shape[0])))

    for index, image in enumerate(images):
        plt.subplot(sqrtn, sqrtn, index+1)
        plt.imshow(image.reshape(28, 28), cmap='gray')
        plt.axis('off')

In [ ]:
model.eval()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Test
with torch.no_grad():
    for images, _ in testloader:
        print("Originals")
        inputs = images.view(-1, 28*28)
        show_images(inputs)
        plt.show()

        print("Produced by AutoEncoder")
        inputs = inputs.to(device)
        code, outputs = model(inputs)
        show_images(outputs.cpu())
        plt.show()
        break

## Visualise compressed data
Earlier, we set the dimension of the output of the encoder to $2$. These two values can be thought of as the $x$ and $y$ coordinates of a point in a two-dimensional space that represents a compressed version of the input image.  
Let's generate a scatter plot of these encoded representations of the images in the test set.


In [ ]:
axis_x = []
axis_y = []
answers = []

# turn off gradient calculation to speed up the process
with torch.no_grad():
    for images, labels in testloader:
        inputs = images.view(-1, 28*28)
        answers += labels.tolist()

        inputs = inputs.to(device)
        code, outputs = model(inputs)
        code = code.cpu()
        axis_x += code[:, 0].tolist()
        axis_y += code[:, 1].tolist()


plt.scatter(axis_x, axis_y, c=answers)
plt.colorbar()
plt.show()

The data compressed by the Autoencoder has been able to grasp the existence of different features in each number.

## One step further: different types of Autoencoders

There are several types of autoencoders, each designed for a specific purpose. Here are some of the most common types:

* **Convolutional autoencoder**: This type of autoencoder is specifically designed for image data. It uses convolutional layers in the encoder and decoder to take advantage of the spatial structure in the data.
* **Recurrent autoencoder**: This type of autoencoder is designed for sequential data, such as time series or text. It uses recurrent neural networks (RNNs) in the encoder and decoder to capture the temporal dependencies in the data.
* **Denoising autoencoder**: This type of autoencoder is designed to remove noise from input data. During training, the model is fed noisy input data and is trained to reconstruct the original, noise-free input.
* **Variational autoencoder** (VAE): This type of autoencoder is designed to generate new data samples that are similar to the original input data. It uses a probabilistic approach to generate data by sampling from a learned distribution.

##### **2. Create a convolutional autoencoder and use it on MNIST. See how the reconstructed images change and how the encoded representations are scattered in a plane.**

Its decoder uses transposed convolutional layers to increase the width and height of the input layers. They work almost exactly the same as convolutional layers, but in reverse. A stride in the input layer results in a larger stride in the transposed convolution layer. For example, if you have a $3x3$ kernel, a $3x3$ patch in the input layer will be reduced to one unit in a convolutional layer. Comparatively, one unit in the input layer will be expanded to a $3x3$ path in a transposed convolution layer. PyTorch provides us with an easy way to create these layers, [`nn.ConvTranspose2d`](https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html).

In [ ]:
# YOUR CODE